In [1]:
import numpy as np
import copy, random, math

slots = 4

enrollments = {
    1: [1,2,3,4,5],
    2: [6,7,8,9], 
    3: [10,11,12],
    4: [1,2,3,4],
    5: [5,6,7,8],
    6: [9,10,11,12],
    7: [1,2,3,5],
    8: [6,7,8],
    9: [4,9,10,11,12],
    10: [1,2,4,5],
    11: [3,6,7,8],
    12: [9,10,11,12]
}

num_enrollments = len(enrollments)

# 4.1 a)
def generate_random_solution():
    return np.random.randint(1, slots + 1, num_enrollments)

# 4.1 b)
def get_num_incompatibilities(discipline_1, discipline_2):
    if (discipline_1 not in enrollments or discipline_2 not in enrollments):
        return 0
    incompatibilities = [student for student in enrollments[discipline_1] if student in enrollments[discipline_2]];
    return len(incompatibilities)
    
# 4.1 c)
def evaluate_solution(solution):
    num_incompatibilities = 0
    for discipline_1 in range(0, len(solution) - 1):
        for discipline_2 in range(discipline_1 + 1, len(solution)):
            if solution[discipline_1] == solution[discipline_2]:
                num_incompatibilities += get_num_incompatibilities(discipline_1 + 1, discipline_2 + 1)
    return -num_incompatibilities

# 4.1 d)
def get_neighbor_solution(solution):
    neighbor_solution = copy.deepcopy(solution)
    discipline_to_mutate = np.random.randint(0, len(solution))
    neighbor_solution[discipline_to_mutate] = (neighbor_solution[discipline_to_mutate] + np.random.randint(0, slots)) % slots 
    return neighbor_solution

# 4.1 e) Hill Climbing
def get_hc_solution(num_iterations):
    iteration = 0;
    best_solution = generate_random_solution() # Best solution after 'num_iterations' iterations without improvement
    best_score = evaluate_solution(best_solution)
    
    print(f"Initial solution: {best_solution}, score: {best_score}")
    
    while iteration < num_iterations:
        iteration += 1
        neighbor_solution = get_neighbor_solution(best_solution)
        neighbor_score = evaluate_solution(neighbor_solution)
        if neighbor_score > best_score:
            iteration = 0
            best_solution = neighbor_solution
            best_score = neighbor_score
            
    print(f"  Final solution: {best_solution}, score: {best_score}")
    return best_solution

# 4.1 e) Simulated Annealing
def get_sa_solution(num_iterations):
    iteration = 0;
    temperature = 1000;
    solution = generate_random_solution() # Best solution after 'num_iterations' iterations without improvement
    score = evaluate_solution(solution)
    
    best_solution = copy.deepcopy(solution)
    best_score = score
    
    print(f"Initial solution: {solution}, score: {score}")
    
    while iteration < num_iterations:
        temperature = temperature * 0.999
        iteration += 1
        neighbor_solution = get_neighbor_solution(best_solution)
        neighbor_score = evaluate_solution(neighbor_solution)
        
        delta = neighbor_score - score 
        
        if delta > 0 or np.exp(-delta/temperature) > random.random():
            solution = neighbor_solution
            score = neighbor_score
            if score > best_score:
                iteration = 0
                best_solution = copy.deepcopy(solution)
                best_score = score 
            
    print(f"  Final solution: {best_solution}, score: {best_score}")
    return best_solution 
            
# 4.2 c)
def crossover_solutions(solution_1, solution_2):
    mid_index = math.trunc(len(solution_1) / 2)
    child_1 = np.append(solution_1[0:mid_index], solution_2[mid_index:]) 
    child_2 = np.append(solution_2[0:mid_index], solution_1[mid_index:])
    return child_1, child_2
    
#4.2 d)
def generate_population(num_population):
    solutions = []
    for i in range(num_population):
        solutions.append(generate_random_solution())
    return solutions
    
def tournament_select(population, tournament_size):
    population_copy = copy.deepcopy(population)
    best_solution = population_copy[0]
    best_score = evaluate_solution(population_copy[0])
    for i in range(tournament_size):
        index = np.random.randint(0, len(population_copy))
        score = evaluate_solution(population_copy[index])
        if score > best_score:
            best_score = score
            best_solution = population_copy[index]
        del population_copy[index]
    return best_solution

def get_greatest_fit(population):
    best_solution = population[0]
    best_score = evaluate_solution(population[0])
    for i in range(1, len(population)):
        score = evaluate_solution(population[i])
        if score > best_score:
            best_score = score
            best_solution = population[i]
    return best_solution, best_score

def replace_least_fittest(population, offspring):
    least_fittest_index = 0
    least_fittest_value = evaluate_solution(population[0])
    for i in range(1, len(population)):
        score = evaluate_solution(population[i])
        if score < least_fittest_value:
            least_fittest_value = score
            least_fittest_index = i
    population[least_fittest_index] = offspring

def roulette_select(population):
    score_sum = sum([evaluate_solution(solution) for solution in population])
    selection_probabilities = [evaluate_solution(solution) / score_sum for solution in population]
    return population[np.random.choice(len(population), p=selection_probabilities)]

#4.2 e)
def mutate_solution_1(solution):
    index_1 = np.random.randint(0, len(solution))
    index_2 = (index_1 + np.random.randint(0, len(solution))) % (len(solution) - 1) # Efficient way to generate a non-repeated index
    solution[index_1], solution[index_2] = solution[index_2], solution[index_1]
    return solution

def mutate_solution_2(solution):
    index = np.random.randint(0, len(solution))
    solution[index] = np.random.randint(1, slots + 1)
    return solution

#4.3 f)       

def genetic_algorithm(num_iterations, num_initial_pop, crossover_func, mutation_func):
    population = generate_population(num_initial_pop)
    
    best_solution = population[0] # Initial solution
    best_score = evaluate_solution(population[0])
    best_solution_generation = 0 # Generation on which the best solution was found
    
    generation_no = 0
    
    print(f"Initial solution: {best_solution}, score: {best_score}")
    
    while(num_iterations > 0):
        
        generation_no += 1
        
        tournment_winner_sol = tournament_select(population, 4)
        roulette_winner_sol = roulette_select(population)
        
        # Next generation
        crossover_sol_1, crossover_sol_2 = crossover_func(tournment_winner_sol, roulette_winner_sol)
    
        if np.random.randint(0, 10) < 5: # 50% chance to perform mutation
            replace_least_fittest(population, mutation_func(crossover_sol_1))
            replace_least_fittest(population, mutation_func(crossover_sol_2))
        else:
            replace_least_fittest(population, crossover_sol_1)
            replace_least_fittest(population, crossover_sol_2)
        
        # Checking the greatest fit among the current population
        greatest_fit, greatest_fit_score = get_greatest_fit(population)
        if greatest_fit_score > best_score:
            best_solution = greatest_fit
            best_score = greatest_fit_score
            best_solution_generation = generation_no
        else:
            num_iterations -= 1
        
    print(f"  Final solution: {best_solution}, score: {best_score}")
    print(f"  Found on generation {best_solution_generation}")
    
    return best_solution
                        
        
""" Tests """        

print("Hill climbing:\n")
get_hc_solution(100)

print("\nSimulated Annealing:\n")
get_sa_solution(100)

print("\nGeneric Algorithm:\n")
genetic_algorithm(100, 30, crossover_solutions, mutate_solution_2)

print("")

Hill climbing:

Initial solution: [2 4 4 4 1 4 2 4 3 2 1 3], score: -25
  Final solution: [2 2 0 4 1 4 1 4 3 0 3 1], score: -1

Simulated Annealing:

Initial solution: [2 2 2 3 2 2 4 4 4 1 1 1], score: -8
  Final solution: [2 3 0 3 0 2 4 4 4 1 1 1], score: 0

Generic Algorithm:

Initial solution: [2 2 2 3 1 4 1 4 1 1 3 4], score: -11
  Final solution: [2 2 2 3 1 4 4 4 1 1 3 3], score: -3
  Found on generation 62

